In [1]:
import numpy as np
from scipy.integrate import quad
from galpy.potential import evaluatePotentials, evaluateDensities

def density_function(r, potential):
    """
    Calculate the density from the spherical potential using the Poisson equation.
    """
    rho = evaluateDensities(potential, r, 0)
    return rho

def integrand(E, psi, dpsi_drho, r, rho):
    """
    Integrand for the Eddington inversion.
    """
    return dpsi_drho / np.sqrt(E - psi)

def eddington_inversion(potential, r_min=1e-5, r_max=100, n_points=1000):
    """
    Perform the Eddington inversion to calculate f(E) for a given spherical potential.
    """
    # Generate a grid of radii and compute potential and density
    r = np.logspace(np.log10(r_min), np.log10(r_max), n_points)
    psi = -evaluatePotentials(potential, r, 0)  # Negative potential for bound systems
    rho = np.array([density_function(ri, potential) for ri in r])

    # Derivative of psi with respect to rho
    dpsi_drho = np.gradient(psi, r)

    # Calculate f(E) using numerical integration
    f_E = []
    for E in psi:
        result, _ = quad(integrand, r_min, r_max, args=(E, psi, dpsi_drho, r, rho))
        f_E.append(result / np.sqrt(8) / np.pi**2)

    return np.array(f_E)

# Example usage with the HernquistPotential
from galpy.potential import HernquistPotential
hernquist = HernquistPotential(amp=1., a=1.)

f_E_values = eddington_inversion(hernquist)

# Plot the results
import matplotlib.pyplot as plt
plt.plot(f_E_values)
plt.xlabel('Energy E')
plt.ylabel('f(E)')
plt.title('Eddington Inversion for Hernquist Potential')
plt.show()


TypeError: integrand() takes 5 positional arguments but 6 were given